In [1]:
from transformers import EncoderDecoderModel, BertTokenizerFast, Seq2SeqTrainer, Seq2SeqTrainingArguments
from zipfile import ZipFile
import os.path as osp

import datasets
from datasets import Dataset

from functools import partial

from typing import List

from utils import load_dataset

import torch
import torch.nn as nn
import torch.nn.functional as F

teacher_processor = EncoderDecoderModel.from_pretrained("kykim/bertshared-kor-base")
tokenizer = BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [2]:
dataset = load_dataset(tokenizer=tokenizer)

In [5]:
dataset['train'][0].keys()

dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'])

In [8]:
def process(examples:datasets.arrow_dataset.Batch):
    
    processed_inputs = teacher_processor(input_ids=torch.tensor(examples['input_ids']),
                                         attention_mask=torch.tensor(examples['attention_mask']))
    return processed_inputs

processed_datasets = dataset.map(process, batched=True, batch_size=4)

  0%|          | 0/70 [00:00<?, ?ba/s]

In [ ]:
class ImageDistilTrainer(Seq2SeqTrainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher = teacher_model
        self.student = student_model
        self.loss_function = nn.KLDivLoss(reduction="batchmean")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.teacher.to(device)
        self.teacher.eval()
        self.temperature = temperature
        self.lambda_param = lambda_param

    def compute_loss(self, student, inputs, return_outputs=False):
        student_output = self.student(**inputs)

        with torch.no_grad():
          teacher_output = self.teacher(**inputs)

        # Compute soft targets for teacher and student
        soft_teacher = F.softmax(teacher_output.logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output.logits / self.temperature, dim=-1)

        # Compute the loss
        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)

        # Compute the true label loss
        student_target_loss = student_output.loss

        # Calculate final loss
        loss = (1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss
        return (loss, student_output) if return_outputs else loss